In [25]:
#Importar librerías
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import networkx as nx
from pyvis.network import Network
from bs4 import BeautifulSoup

## PARÁMETROS

In [26]:
# Filtrar por tabla:
esquema_filtro = None # En el caso no haya filtro de debe poner None
tabla_filtro = None # En el caso no haya filtro de debe poner None
if esquema_filtro == None or tabla_filtro == None:
    filtro = None
else:
    filtro = esquema_filtro + '_' + tabla_filtro

# ------------------------------------------------------------------------------------------------------------------

# Filtrar esquema en general
esquema_filtro = None  # En el caso no haya filtro de debe poner None

## CONEXIÓN SQL SERVER - AUTENTICACIÓN WINDOWS

In [27]:
##configuración de la conección
server = 'PE3443175W2'
database = 'Base_Grafos_Test'
trusted_connection = 'yes' # esto indica que se usará la autentificación de windows
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection={trusted_connection}'
conn = pyodbc.connect(connection_string)
engine  = create_engine(f'mssql+pyodbc:///?odbc_connect={connection_string}')
cursor = conn.cursor()

In [28]:
if esquema_filtro == None:
    query_tablas = """
        SELECT SCHEMA_NAME(schema_id) as schema_name, name as table_name , sum(rows) as row_count
        FROM sys.tables t
        inner join sys.partitions p ON t.object_id = p.object_id
        where p.index_id IN (0,1) 
        group by SCHEMA_NAME(schema_id),name"""
else:
    query_tablas = f"""
        SELECT SCHEMA_NAME(schema_id) as schema_name, name as table_name , sum(rows) as row_count
        FROM sys.tables t
        inner join sys.partitions p ON t.object_id = p.object_id
        where p.index_id IN (0,1) and SCHEMA_NAME(schema_id) = '{esquema_filtro}'
        group by SCHEMA_NAME(schema_id),name"""

In [29]:
#Obtener la lista de las tablas en la BD
tabla_cursor = conn.cursor()
tabla_cursor.execute(query_tablas)
tablas = tabla_cursor.fetchall()
tablas

[('dbo', 'TClientes', 15),
 ('dbo', 'TEmpleados', 5),
 ('dbo', 'TFacturas', 3),
 ('dbo', 'TPedidos', 4),
 ('dbo', 'TProductos', 3),
 ('dbo', 'TVacaciones', 4)]

In [30]:
#Diccionario para guardar el nombre de los df's
dataframes = {}
dataframes_vacios = {}

#Iterar sobre las tablas y cargar cada una a un df 
for tabla in tablas:
    esquema_tabla = tabla[0]
    nombre_tabla = tabla[1]
    cantidad_filas = tabla[2]
    query = f"SELECT top (10000) * FROM [{esquema_tabla}].[{nombre_tabla}]"

    #Cragar los datos 
    df = pd.read_sql(query, engine)

    #Nombre de la tabla
    nombre_df = esquema_tabla + '_' + nombre_tabla

    #Cambiar y guardar el nombre del df
    globals()[nombre_df] = df.copy()

    #Agregar nombre del df en el diccionario
    if cantidad_filas > 0:
        dataframes[nombre_df] = globals()[nombre_df]
    else:
        dataframes_vacios[nombre_df] = globals()[nombre_df]
    
    del df #Eliminar df

In [31]:
#cursor = conn.cursor()
conn.close()

## CREACIÓN DE GRAFOS PARA OBTENER LAS RELACIONES ENTRE TABLAS

In [32]:
# Crear un grafo dirigido para representar las relaciones
grafo_relaciones = nx.DiGraph()

In [33]:
# Definir función para calcular el porcentaje de similitud de los datos
def porcentaje_valores_en_comun(df1, df2, columna_df1, columna_df2):
    df1[columna_df1] = df1[columna_df1].astype(str)
    df2[columna_df2] = df2[columna_df2].astype(str)

    valores_unicos_df1 = set(df1[columna_df1])
    valores_unicos_df2 = set(df2[columna_df2])

    valores_en_comun = valores_unicos_df1.intersection(valores_unicos_df2)
    porcentaje_en_comun = len(valores_en_comun) / len(valores_unicos_df1) * 100

    return porcentaje_en_comun

In [34]:

# Definir una función para inferir relaciones basándose en la similitud de Jaccard
def inferir_relaciones(df1, df2, umbral_similitud=70):
    campos_relacionados = []
    nombre_df1 = [name for name, var in globals().items() if var is df1][0]
    nombre_df2 = [name for name, var in globals().items() if var is df2][0]
    for col1 in df1.columns:
        if 'float' in str(df1[col1].dtype):
            continue
        if len(df1[col1]) == df1[col1].nunique():
            for col2 in df2.columns:
                if 'float' in str(df2[col2].dtype):
                    continue
                if porcentaje_valores_en_comun(df1, df2, col1, col2) > umbral_similitud:
                    campos_relacionados.append((nombre_df1 + '.' + col1, nombre_df2 + '.' + col2))
                elif porcentaje_valores_en_comun(df2, df1, col2, col1) > umbral_similitud:
                    campos_relacionados.append((nombre_df2 + '.' + col2, nombre_df1 + '.' + col1))
        else:
            for col2 in df2.columns:
                if 'float' in str(df2[col2].dtype):
                    continue
                if len(df2[col2]) == df2[col2].nunique():
                    if porcentaje_valores_en_comun(df1, df2, col1, col2) > umbral_similitud:
                        campos_relacionados.append((nombre_df1 + '.' + col1, nombre_df2 + '.' + col2))
                    elif porcentaje_valores_en_comun(df2, df1, col2, col1) > umbral_similitud:
                        campos_relacionados.append((nombre_df2 + '.' + col2, nombre_df1 + '.' + col1))
    if len(campos_relacionados) > 0:
        return campos_relacionados
    else:
        return None

In [35]:
def relaciones_entre_tablas(diccionario_tablas, tabla_especifica=None):
    # Crear un grafo dirigido para representar las relaciones
    grafo_relaciones = nx.DiGraph()
    # Inicializar el conjunto de nombres de DataFrames relacionados
    dataframes_relacionados = set()
    relaciones_comparadas = set()
    if tabla_especifica == None:
        # Iterar sobre los pares de DataFrames y buscar relaciones
        for nombre_df1, df1 in diccionario_tablas.items():
            for nombre_df2, df2 in diccionario_tablas.items():
                if nombre_df1 != nombre_df2:
                    relaciones = inferir_relaciones(df1, df2)
                    if relaciones:
                        grafo_relaciones.add_edge(nombre_df1, nombre_df2, relaciones=relaciones)
                        dataframes_relacionados.add(nombre_df1)
                        dataframes_relacionados.add(nombre_df2)
                        relaciones_comparadas.add((nombre_df1,nombre_df2))

        # Encontrar DataFrames sin relaciones y agregar nodos para ellos
        dataframes_sin_relacion = set(diccionario_tablas.keys()) - dataframes_relacionados
        for df in dataframes_sin_relacion:
            grafo_relaciones.add_node(df)

        return grafo_relaciones

    else:
        for nombre_df1, df1 in diccionario_tablas.items():
            for nombre_df2, df2 in diccionario_tablas.items():
                if nombre_df1 != nombre_df2:
                    if nombre_df1 == tabla_especifica or nombre_df2 == tabla_especifica:
                        relaciones = inferir_relaciones(df1, df2)
                        if relaciones:
                            grafo_relaciones.add_edge(nombre_df1, nombre_df2, relaciones=relaciones)
                            dataframes_relacionados.add(nombre_df1)
                            dataframes_relacionados.add(nombre_df2)
                            relaciones_comparadas.add((nombre_df1,nombre_df2))
                            
        return grafo_relaciones


In [36]:
def relacionEntreTablas(tabla_1, tabla_2):
    relaciones_entre_tablas = grafo_relaciones.get_edge_data(tabla_1, tabla_2)
    if relaciones_entre_tablas:
        relaciones = relaciones_entre_tablas.get('relaciones')
        if relaciones:
            return 1
        else:
            return 0
    else:
        return 0

In [37]:
def eliminarRelaciones(relaciones_a_eliminar):
    tablas_afectadas = set()
    for _ in range(2):
        for relacion in relaciones_a_eliminar:
            nodo_inicio, campo_inicio, nodo_fin, campo_fin = relacion
            if grafo_relaciones.has_edge(nodo_inicio, nodo_fin):
                relaciones_actuales = grafo_relaciones[nodo_inicio][nodo_fin]['relaciones']
                relaciones_actualizadas = [r for r in relaciones_actuales if r != (nodo_inicio + '.' + campo_inicio, nodo_fin + '.' + campo_fin)]
                grafo_relaciones[nodo_inicio][nodo_fin]['relaciones'] = relaciones_actualizadas
                tablas_afectadas.add(nodo_inicio)
                tablas_afectadas.add(nodo_fin)

                relaciones_actuales = grafo_relaciones[nodo_inicio][nodo_fin]['relaciones']
                relaciones_actualizadas = [r for r in relaciones_actuales if r != (nodo_fin + '.' + campo_fin, nodo_inicio + '.' + campo_inicio)]
                grafo_relaciones[nodo_inicio][nodo_fin]['relaciones'] = relaciones_actualizadas
                tablas_afectadas.add(nodo_inicio)
                tablas_afectadas.add(nodo_fin)
        relaciones_a_eliminar = [r_eliminar[2:] + r_eliminar[:2] for r_eliminar in relaciones_a_eliminar]
    # Eliminar todas las relaciones entre las tablas afectadas
    for tabla_inicio in tablas_afectadas:
        for tabla_fin in tablas_afectadas:
            try:
                if relacionEntreTablas(tabla_inicio, tabla_fin) == 0 and tabla_inicio != tabla_fin:
                    grafo_relaciones.remove_edge(tabla_inicio, tabla_fin)
            except nx.NetworkXError:
                pass
    print('---------------------------------Completado con éxito---------------------------------')

In [38]:
def agregarRelaciones(relaciones_a_agregar):
    for relacion in relaciones_a_agregar:
        nodo_inicio, campo_inicio, nodo_fin, campo_fin = relacion
        if relacionEntreTablas(nodo_inicio, nodo_fin) == 0:
            grafo_relaciones.add_edge(nodo_inicio, nodo_fin, relaciones=[(nodo_inicio + '.' + campo_inicio, nodo_fin + '.' + campo_fin)])
            grafo_relaciones.add_edge(nodo_inicio, nodo_fin)  # Agregar relación entre tablas automáticamente
        else:
            relaciones_actuales = grafo_relaciones[nodo_inicio][nodo_fin]['relaciones']
            relaciones_actuales.append((nodo_inicio + '.' + campo_inicio, nodo_fin + '.' + campo_fin))
            grafo_relaciones[nodo_inicio][nodo_fin]['relaciones'] = relaciones_actuales
            grafo_relaciones[nodo_fin][nodo_inicio]['relaciones'] = relaciones_actuales
    print('---------------------------------Completado con éxito---------------------------------')

In [39]:
if filtro == None:
    grafo_relaciones = relaciones_entre_tablas(dataframes)
else:
    grafo_relaciones = relaciones_entre_tablas(dataframes,filtro)

In [40]:
# Eliminar relaciones
df_eliminar_relaciones = pd.read_excel('Modificar_Relaciones.xlsx',sheet_name='Eliminar')

if len(df_eliminar_relaciones) > 0:
    relaciones_eliminar = []
    for index, row in df_eliminar_relaciones.iterrows():
        esquema_1, tabla_1, campo_1 = row['Esquema_1'], row['Tabla_1'], row['Campo_1']
        esquema_2, tabla_2, campo_2 = row['Esquema_2'], row['Tabla_2'], row['Campo_2']
        relacion_eliminar = (f'{esquema_1}_{tabla_1}', campo_1, f'{esquema_2}_{tabla_2}', campo_2)
        relaciones_eliminar.append(relacion_eliminar)
    eliminarRelaciones(relaciones_eliminar)

---------------------------------Completado con éxito---------------------------------


In [41]:
# Agregar relaciones
df_agregar_relaciones = pd.read_excel('Modificar_Relaciones.xlsx',sheet_name='Agregar')

if len(df_agregar_relaciones) > 0:
    relaciones_agregar = []
    for index, row in df_agregar_relaciones.iterrows():
        esquema_1, tabla_1, campo_1 = row['Esquema_1'], row['Tabla_1'], row['Campo_1']
        esquema_2, tabla_2, campo_2 = row['Esquema_2'], row['Tabla_2'], row['Campo_2']
        relacion_agregar = (f'{esquema_1}_{tabla_1}', campo_1, f'{esquema_2}_{tabla_2}', campo_2)
        relaciones_agregar.append(relacion_agregar)
    agregarRelaciones(relaciones_agregar)

In [46]:
# Dibujar el grafo

# Obtener las posiciones una vez que se han agregado todos los nodos y aristas
posiciones = nx.spring_layout(grafo_relaciones,seed=42)

# Crear un objeto de red para visualización interactiva
net = Network(notebook=True, height="700px", width="100%")

# Agregar nodos al objeto de red con nombres y posiciones
for nodo, pos in posiciones.items():
    net.add_node(nodo, label=nodo, x=pos[0] * 100, y=pos[1] * 100)

# Agregar aristas al objeto de red con tooltips
for edge in grafo_relaciones.edges():
    relaciones = grafo_relaciones[edge[0]][edge[1]]['relaciones']
    if relaciones:
        tooltip_text = f'Relaciones ({len(relaciones)})' +  ':\n{}'.format('\n'.join([f"({x[0]}, {x[1]})" for x in relaciones]))
    else:
        tooltip_text = 'No hay relaciones disponibles'
    net.add_edge(edge[0], edge[1], title=tooltip_text)       

# Guardar la visualización interactiva
net.save_graph("Entidad_Relación.html")

# Abrir el archivo HTML con codificación UTF-8
with open("Entidad_Relación.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")

# Crear un encabezado h1 con el título
header_tag = soup.new_tag("h1")
header_tag.string = "MODELO ENTIDAD-RELACIÓN"
header_tag['style'] = 'font-size: 36px;'  # Ajustar el tamaño de la fuente según sea necesario
soup.body.insert(0, header_tag)  # Insertar el encabezado al principio del cuerpo del HTML

# Guardar el HTML actualizado con el encabezado y la codificación correcta
with open("Entidad_Relación.html", "w", encoding="utf-8") as file:
    file.write(str(soup))

In [43]:
df_relaciones = pd.DataFrame(columns=['Esquema_1', 'Tabla_1', 'Campo_1', 'Esquema_2', 'Tabla_2', 'Campo_2'])
l_relaciones = []
# Mostrar todas las relaciones con sus campos correspondientes
for edge in grafo_relaciones.edges(data=True):
    nodo_inicio = edge[0]
    nodo_fin = edge[1]
    relaciones = edge[2]['relaciones'] if 'relaciones' in edge[2] else None
    if relaciones:
        for relacion in relaciones:
            rela_1_split = relacion[0].split('_', 1)
            rela_2_split = relacion[1].split('_', 1)
            tabla_campo_1_split = rela_1_split[1].split('.')
            tabla_campo_2_split = rela_2_split[1].split('.')
            esquema1, tabla1, campo1 = rela_1_split[0], tabla_campo_1_split[0], tabla_campo_1_split[1]
            esquema2, tabla2, campo2 = rela_2_split[0], tabla_campo_2_split[0], tabla_campo_2_split[1]
            l_relacion = [esquema1, tabla1, campo1, esquema2, tabla2, campo2]
            df_relaciones.loc[len(df_relaciones)] = l_relacion

df_relaciones.drop_duplicates(inplace=True)

df_relaciones.to_excel('Relaciones.xlsx',index=False)

df_relaciones

,Esquema_1,Tabla_1,Campo_1,Esquema_2,Tabla_2,Campo_2
0,dbo,TPedidos,ClienteID_2P,dbo,TClientes,ClienteID_keyC
2,dbo,TPedidos,PedidoIDP,dbo,TFacturas,PedidoIDF
3,dbo,TPedidos,Producto,dbo,TProductos,ProductoP
4,dbo,TVacaciones,IDEmpleadoV,dbo,TEmpleados,EmpleadoIDE
5,dbo,TFacturas,PedidoIDF,dbo,TPedidos,PedidoIDP
7,dbo,TProductos,ProductoP,dbo,TPedidos,Producto
